# Caso Práctico: Detección de SPAM

## Conjunto de datos: Detección de SPAM

Este conjunto de datos está formado por diversos emails, de los cuales disponemos tanto del texto, asunto, correo de emisores, receptores y fechas de emisión.

Disponemos de dos bases de datos, una está compuesta sólo de correos considerados spam, la segunda de correos considerados buenos.

## Lectura de datos

In [1]:
import os

spam_emails_path = os.path.join("spamassassin-public-corpus", "spam")
ham_emails_path = os.path.join("spamassassin-public-corpus", "ham")
labeled_file_directories = [(spam_emails_path, 0), (ham_emails_path, 1)]

Recorremos los documentos listados extrayendo el texto que será analizado y procesado para extraer información del mismo.

In [3]:
email_corpus = []
labels = []

for class_files, label in labeled_file_directories:
    files = os.listdir(class_files)
    for file in files:
        file_path = os.path.join(class_files, file)
        try:
            with open(file_path, "r") as currentFile:
                email_content = currentFile.read().replace("\n", "")
                email_content = str(email_content)
                email_corpus.append(email_content)
                labels.append(label)
        except:
            pass

## División del conjunto de dato en train y test
Aplicamos la técnica aprendida de división en train y test que reduce la posibilidad de sesgo.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    email_corpus, labels, test_size=0.2, random_state=11
)

## Procesado de datos, convertimos texto en vectores

En este caso se ha aplicado la técnica de vectorizado por hashing.
Y posteriormente tfidf para buscar similitudes entre palabras en función de sus palabras vecinas en frases y párrafos.

## Entrenamiento de un modelo de árbol de decisión
Se trata de un modelo de clasificació binaria en el que las clases están balanceadas.
Se consideran las clases balanceadas.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn import tree

nlp_followed_by_dt = Pipeline(
    [
        ("vect", HashingVectorizer(input="content", ngram_range=(1, 3))),
        ("tfidf", TfidfTransformer(use_idf=True,)),
        ("dt", tree.DecisionTreeClassifier(class_weight="balanced")),
    ]
)
nlp_followed_by_dt.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 HashingVectorizer(alternate_sign=True, analyzer='word',
                                   binary=False, decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, n_features=1048576,
                                   ngram_range=(1, 3), norm='l2',
                                   preprocessor=None, stop_words=None,
                                   strip_accents=None,
                                   token_pattern='(?u)\\b\\w\\w+\\b',
                                   tokenizer=None...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('dt',
                 DecisionTreeClassifier(class_weight='balanced',
                                        criterion='gini', max_dept

## Evaluación

In [6]:
print(nlp_followed_by_dt.score(X_train, y_train))

1.0


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_test_pred = nlp_followed_by_dt.predict(X_test)
print("Acuracy", accuracy_score(y_test, y_test_pred))
print("Matriz de confusión", confusion_matrix(y_test, y_test_pred))